In [1]:
import os 

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/jayraj2498/DSP_End_KNN_prj.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "jayraj2498"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "777e2be0b0c43fcc2efbc898716cbaebe35c912b "




In [10]:
%pwd

'e:\\Ds_p1\\DSP_End_KNN_prj'

In [5]:
os.chdir("../")

In [6]:
%pwd

'e:\\Ds_p1\\DSP_End_KNN_prj'

In [11]:
from dataclasses import dataclass 
from pathlib import Path 


@dataclass 
class ModelEvaluationConfig : 
    root_dir : Path 
    test_data_path : Path 
    model_path : Path 
    all_params : dict 
    metric_file_name : Path 
    target_column : str 
    mlflow_uri : str

In [12]:
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml , create_directories , save_json 

In [ ]:
class ConfigurationManager : 
    
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH): 
        
        self.config =read_yaml(config_filepath) 
        self.params= read_yaml(params_filepath) 
        self.schema= read_yaml(schema_filepath) 
        
        create_directories([self.config.artifacts_root]) 
        
        
    def get_model_evaluation_config(self)-> ModelEvaluationConfig : 
        config = self.config.model_evaluation 
        params = self.params.ElasticNet 
        schema = self.schema.TARGET_COLUMN  
        
        create_directories([config.root_dir]) 
        
        model_evaluation_config =  ModelEvaluationConfig(
            root_dir = config.root_dir ,
            test_data_path = config.test_data_path , 
            model_path = config.model_path , 
            all_params = params , 
            metric_file_name = config.metric_file_name , 
            target_name = schema.name )

NameError: name 'CONFIG_FILE_PATH' is not defined